In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab\ Notebooks/transfer_learning/
# %cd /content/drive/Othercomputers/My Mac/transfer_learning/

/content/drive/.shortcut-targets-by-id/1FbM89xH1dKPtK3SG5wd4NoMu_YTXTHzo/transfer_learning


In [3]:
import json
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy


for model_name in ['', 'moco2_200ep_', 'moco3_300ep_', 'moco3_1000ep_']:
# for model_name in ['moco2_200ep_', 'moco3_300ep_', 'moco3_1000ep_']:
  summarized_data = {
    'noise': 0,
    'pretrained model': model_name if model_name != '' else 'moco2_800ep_',
    'total epochs': 10,
    'batch size': 64,
    'data': [],
  }
  if model_name == 'moco2_200ep_' or model_name == 'moco3_1000ep_':
    x_arr = ['lpfull', 'lp7ft3', 'lp5ft5', 'lp3ft7', 'ft']
  else:
    x_arr = ['lpfull', 'lp7ft3', 'lpftyes', 'lp3ft7', 'ft']
  for x, lp_eps in zip(x_arr, [10, 7, 5, 3, 0]):
    best_lr = 0.025
    best_accs = np.zeros(3)
    best_lognames = None
    for lr in [0.025, 0.05, 0.1]:
      accs = []
      lognames = []
      for seed in range(3):
        try:
          with open('logs/'+model_name+f'{x}_cifar10_lr_{lr}_nm_0_mt_0_sd_{seed}/best.json', 'r') as file:
            js_data = json.load(file)
            accs.append(js_data['best_test_acc/cifar10-test'])
          lognames.append(model_name+f'{x}_cifar10_lr_{lr}_nm_0_mt_0_sd_{seed}/best.json')
        except Exception as e:
          print(model_name+f'{x}_cifar10_lr_{lr}_nm_0_mt_0_sd_{seed}/best.json')
      if np.mean(accs) > np.mean(best_accs):
        best_lr = lr
        best_accs = deepcopy(accs)
        best_lognames = deepcopy(lognames)
    summarized_data['data'].append(
        {
            'lp epochs': lp_eps,
            'seeds': [0, 1, 2],
            'learning rate': best_lr,
            'test accuracies': best_accs,
            'log names': best_lognames,
        }
    )
  if model_name == '':
    model_name = 'moco2_800ep_'
  with open('logs/'+model_name+'nm0.json', 'w') as file:
    json.dump(summarized_data, file)
